In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
#export
import sys
sys.path.append('/data/home/jdavey/notebooks/fastai/mi')
from nn.neural import Neural

In [3]:
#export
import torch

In [4]:
#export
class SoftmaxCrossEntropy(Neural):
    
    def softmax_forward(self, inp): 
        m = inp.max(-1)[0]
        exps = (inp-m[:, None]).exp()
        return exps / exps.sum(-1,keepdim=True)
    
    def cross_entropy_forward(self, inp, targ):
         return -inp[range(targ.shape[0]), targ].mean()
    
    def forward(self, inp_softmax, targ):
        inp = self.softmax_forward(inp_softmax)
        return self.cross_entropy_forward(inp, targ)
    
    def bwd(self, out, inp, targ): 
        one_hot = torch.zeros(inp.shape)
        one_hot[torch.arange(targ.shape[0]), targ] = 1
        
        m = targ.shape[0]
        grad = self.softmax_forward(inp)
        grad[torch.arange(m), targ] = -1
        inp.g = grad/m

In [6]:
!python notebook2script.py nn_softmaxCrossEntropy.ipynb

Replacing old file
Converted nn_softmaxCrossEntropy.ipynb to /data/home/jdavey/notebooks/fastai/mi/nn/softmaxCrossEntropy.py


$$\hbox{softmax(x)}_{i} = \frac{e^{x_{i}}}{\sum_{0 \leq j \leq n-1} e^{x_{j}}}$$ 

Cross entropy loss

$$ -\sum x\, \log p(x) $$

 There is a way to compute the log of the sum of exponentials in a more stable way, called the [LogSumExp trick](https://en.wikipedia.org/wiki/LogSumExp). The idea is to use the following formula:

$$\log \left ( \sum_{j=1}^{n} e^{x_{j}} \right ) = \log \left ( e^{a} \sum_{j=1}^{n} e^{x_{j}-a} \right ) = a + \log \left ( \sum_{j=1}^{n} e^{x_{j}-a} \right )$$